TERMOS RESERVADOS - NENHUM ESTADO PODE CONTER OS SEGUINTES SÍMBOLOS:

-> &
-> _

In [1]:
#Imports
import re
import pprint

In [2]:
def txtToAutomato(filename):
        file = open(filename, 'r')
        content = file.readlines()
        file.close()
        
        af = Automato()
        
        #contrucao da matriz a armazena funcao programa
        i = len(content) - 2
        af.fPrograma = [[0 for x in range(3)] for y in range(i)]
        
        #content[0]
        tmp = re.split("=", content[0])
        nome = tmp[0] #pega nome do automato
        defAutomato = re.findall('({[A-Za-z0-9,]+})',tmp[1])

        #remove colchetes
        af.estados = re.findall('[A-Za-z0-9]+', defAutomato[0]) 
        af.alfabeto = re.findall('[A-Za-z0-9]+', defAutomato[1])
        af.estadosFinais = re.findall('[A-Za-z0-9]+', defAutomato[2])
            
        tmp2 = re.findall('(},[A-Za-z0-9]+,{)',tmp[1])
        af.estadoInicial = re.findall('([A-Za-z0-9]+)',tmp2[0])
        af.estadoInicial = af.estadoInicial[0]
        
        i = 0
        for c in range(2, len(content)): #comeca em 2 pq a linha 1 é 'prog'
            tmp = re.split('=', content[c])
            tmp2 = re.findall('([A-Za-z0-9]{1,})', tmp[0]) #pega (q,s)
            
            af.fPrograma[i][0] = tmp2[0]     #pega estado lido
            af.fPrograma[i][1] = tmp2[1]     #pega simbolo lido
            tmp2 = re.findall('[^\\n]+', tmp[1])      #pega prox estado (removendo \n)
            af.fPrograma[i][2] = tmp2[0]
            
            i += 1
        
        return af

def csvToList(filename):
    file = open(filename, 'r')
    content = file.readLines()
    file.close()
    
    wordList = re.split(',', content[0])
    
    return wordList

def removeDuplicates(duplicate): 
    final_list = [] 
    for num in duplicate: 
        if num not in final_list: 
            final_list.append(num) 
    return final_list 

def stringMaiorTamanho(lista):
    indexMaior = 0
    tamMaior = 0
    
    for i in range(len(lista)):
        if len(lista[i]) > tamMaior:
            indexMaior = i
            tamMaior = len(lista[i])
    return lista[indexMaior]

def listaMaiorTamanho(lista):
    indexMaior = 0
    tamMaior = 0
    
    for i in range(len(lista)):
        if len(lista[i]) > tamMaior:
            indexMaior = i
            tamMaior = len(lista[i])
    return lista[indexMaior]

#retorna lista de maior tamanho sem elem q ja foram visitados (se tds listas tiverem elemento visitado, retorna a primeira)
def listaMaiorTamanho_NãoRepetido(lista, visitados):
    if not lista:
        return None
    
    indexMaior = 0
    tamMaior = 0
    
    for i in range(len(lista)):
        igual = False
        
        if len(lista[i]) > tamMaior:
            indexMaior = i
            tamMaior = len(lista[i])
        
        elif len(lista[i]) == tamMaior:
            for item in lista[i]:
                if visitados.get(item) != None:
                    igual = True
                    break
                    
            if not igual:
                indexMaior = i
                tamMaior = len(lista[i])
    return lista[indexMaior]
    

In [54]:
#OBS: & Sera considerado simbolo que representa palavra vazia / indefinição

class Automato:
    def __init__(self):
        self.alfabeto = []
        self.estados = []
        self.estadoInicial = ''
        self.estadosFinais = []
        self.fPrograma = [] # matriz com colunas sendo estado, simbolo e prox estado
    
    
    def afnToAfd(self):
        afd = Automato()
        afd.alfabeto = self.alfabeto
        afd.estadoInicial = self.estadoInicial
        afd.fPrograma = [['', '', '']]
        mudou = False
        actualStates = [self.estadoInicial] #Armazena estados atuais que estão sendo analisados
        indice = 0 #Indice atual da fPrograma
        visitados = {}
        estadosAlterados = {}
        
        #-----------------------------------------------
        #Garante as 4 primeiras execuções e para qdo a função programa dos simbolos anteriores não alterar
        while (indice <= 2*len(self.alfabeto) or mudou) and actualStates != None:
            aux = []
            
            #Percorre todos os simbolos do alfabeto
            for symbol in self.alfabeto:
                #print('symbol: ',symbol)
                #print('actualStates: ', actualStates)
                aux2 = []
                #vazial = False
                
                # Percorre tds estados atuais (no inicio é só o q0)
                #pprint.pprint(actualStates)
                for state in actualStates:
                    #print('state: ',state)
                    
                    #pprint.pprint(actualStates)
                    state_fPrograma = self.funcaoPrograma(state,symbol)
                    #print(state_fPrograma)
                    #print('fp ',state_fPrograma)
                    
                    #Se for palavra vazia, ignora ela
                    if state_fPrograma[0] == '&':
                        #state_fPrograma = ''
                        continue
                    
                    #aux.append(state_fPrograma) #pega tds estados
                    for i in range(len(state_fPrograma)):
                        if estadosAlterados.get(state_fPrograma[i]) != None:
                            state_fPrograma[i] = estadosAlterados[state_fPrograma[i]]
                    aux2.extend(state_fPrograma)
                    #print('state: ',state)
                    #print('aux2 ',aux2)
                    
                    
                               
                #TESTE - É NECESSARIO?!
                actualStates.sort()
                aux2.sort()
                #FIMTESTE
                
                #print("a2: ",aux2)
                #print("removendo duplicados aux2")
                aux2 = removeDuplicates(aux2)
                #print('aux2 ',aux2)
                
                jaPercorrido = False
                for item in aux2:
                    if visitados.get(item) != None: #só pega estados que nao foram percorridos
                        jaPercorrido = True
                        break
                    #elif estadosAlterados.get(item):
                        #aux2.replace(item, estadosAlterados[item])
                        
                #for i in range(len(aux2))
                
                if not jaPercorrido:
                    aux.append(aux2)
                    
                
                #print("a1: ",aux)
                #aux2 = stringMaiorTamanho(aux2)
                
                # Atualiza o estado atual na fProg como sendo uma string q é união dos estados gerados na funcao programa
                #for i in range(0, len(actualStates)):
                #    afd.fPrograma[indice][0] += actualStates[i]
                if aux2:
                    #print('as ANTES FOR: ', actualStates)
                    for s in actualStates:
                        afd.fPrograma[indice][0] += s
                        #print('s ', s)
                        #print('afd.fPrograma[indice][0] ', afd.fPrograma[indice][0])
                        #print(actualStates[i])

                    afd.fPrograma[indice][1] = symbol

                    for s in aux2:
                        afd.fPrograma[indice][2] += s

                    #Atualiza estados (se necessario)
                    if afd.estados.count(afd.fPrograma[indice][0]) == 0:
                        afd.estados.append(afd.fPrograma[indice][0])

                    if afd.estados.count(afd.fPrograma[indice][2]) == 0:
                        afd.estados.append(afd.fPrograma[indice][2])

                    # Determina se é estado final (se sim, anexa nos estados finais)
                    for s in actualStates:
                        if self.isFinal(s) and afd.estadosFinais.count(afd.fPrograma[indice][0]) == 0:
                            afd.estadosFinais.append(afd.fPrograma[indice][0])
                            break #garante que só anexa 1 vez o estado
   
                # Prepara a função programa para a próxima iteração
                afd.fPrograma.append(['', '', ''])
                indice += 1 #incrementa o indice (pq vai para proximo simbolo)
            
            #verificou todos os simbolos para cada estado nos actualStates
            #elimina append fPrograma
            indice -= 1
            afd.fPrograma.pop()
            
            
            #afd.estados.append(afd.fPrograma[indice-1][0])
            
            #Atualiza actual states          
            # actualStates = conj uniao dos estados gerados na funcao programa anterior
            #print("aux ",aux)
            #actual states = conj uniao dos estados gerados na funcao programa anterior NAO VISITADO!
            #visitados[afd.fPrograma[indice-1][0]] = True #afd.fPrograma[indice-1][0] = actualStates concatenado
            visitados[afd.fPrograma[indice][0]] = True #afd.fPrograma[indice-1][0] = actualStates concatenado
            
            for s in actualStates:
                #estadosAlterados[s] = afd.fPrograma[indice-1][0]
                estadosAlterados[s] = afd.fPrograma[indice][0]
            
            actualStates = listaMaiorTamanho_NãoRepetido(aux, visitados)
            #if actualStates == None:
            #    break
            
            #if visitado.get(actualStates[0]) != None:
            #print("as: ", actualStates)
            
            # Verifica se mudou estados (condição de parada do loop)
            if indice > 2*len(self.alfabeto):
                mudou = False
                tamAlfabeto = len(self.alfabeto)
                
                for j in range(0, tamAlfabeto):
                    if afd.fPrograma[indice-j][2] != afd.fPrograma[indice-j-tamAlfabeto][2]: # indice - 1 pq indice = fPrograma da prox iteração (vazia no momento)                       
                        mudou = True
                        break
                
                if mudou == False: #se nao mudou, elimina da fPrograma ultimos resultados (ja q são iguais aos anteriores)
                    for j in range(0, len(self.alfabeto)):
                        afd.fPrograma.pop(indice-j)
                        #afd.fPrograma.pop()
                    
                else: #se mudou, prepara proxima iteração
                    afd.fPrograma.append(['', '', ''])
                    indice += 1
            else:
                afd.fPrograma.append(['', '', ''])
                indice += 1
            
        #-----------------------------------------------
        #elimina fPrograma vazia no final (se houver)
        if afd.fPrograma[len(afd.fPrograma)-1][0] == '' or afd.fPrograma[len(afd.fPrograma)-1][1] == '' or afd.fPrograma[len(afd.fPrograma)-1][2] == '':
            afd.fPrograma.pop()
        
        #fPrograma pronta
        # Retira fPrograma vazia no final
        #afd.fPrograma.pop()
        #print("est inicial")
        #pprint.pprint(afd.estadoInicial)
        #print("est finais")
        #pprint.pprint(afd.estadosFinais)
        #print("fP")
        #pprint.pprint(afd.fPrograma)
        
        afd.estadosFinais = removeDuplicates(afd.estadosFinais)
        afd.estadosFinais.sort()
        
        afd.estados = removeDuplicates(afd.estados)
        afd.estados.sort()
        
        
        # Renomeação de estados
        index = 0
        for state in afd.estados:
            
            newState = 'Q' + str(index)
            afd.estados[index] = newState
            index += 1
            
            # Renomeia na funcao programa
            for i in range(len(afd.fPrograma)):
                if afd.fPrograma[i][0] == state:
                    afd.fPrograma[i][0] = newState
                if afd.fPrograma[i][2] == state:
                    afd.fPrograma[i][2] = newState
            
            # Renomeia estados finais
            for i in range(len(afd.estadosFinais)):
                if state == afd.estadosFinais[i]:
                    afd.estadosFinais[i] = newState
            
            # Atualiza estado inicial
            if state == afd.estadoInicial:
                afd.estadoInicial = newState
        
        return afd           
    
    
    #FPT : Função Programa Total
    def convertAfdToFpt(self):
        d = '&&' #representa o 'd' do ppt da aula (não da para um automato ter esse simbolo no input do arquivo)
        #usar dois &&
        
        naoTotal = False
        i = 0
        fProgramaAux = []
        
        for state in self.estados:
            for symbol in self.alfabeto:
                fp = self.funcaoPrograma(state, symbol)
                if fp[0] == '&':
                    print(state, " ", symbol, " = &")
                    naoTotal = True
                    self.fPrograma.insert(i, [state, symbol, d])
                i += 1
    
        self.fPrograma.sort()
        
        if naoTotal:
            self.estados.append(d)
            for symbol in self.alfabeto:
                self.fPrograma.append([d, symbol, d])
        
        
         
        
    #funcao q retorna lista de estados que é possivel ir diretamente a partir de um (se nao achar, retorna None)
    def estadosAtingiveis(self, estado):
        estAtingiveis = []
        
        for i in range(len(self.fPrograma)):
            if self.fPrograma[i][0] == estado:
                estAtingiveis.append(self.fPrograma[i][2])
                
        if not estAtingiveis:
            estAtingiveis = None
        else:
            estAtingiveis = removeDuplicates(estAtingiveis)
            
        return estAtingiveis
    
    
    #retorna todos os estados alcançaveis a partir de um
    def estadosAlcancaveis(self, estado):
        estAlcancaveis = []
        pilha = []
        visitados = {}
        pilha.append(estado)
        
        while pilha:
            est = pilha.pop()
            
            for i in range(len(self.fPrograma)):
                if self.fPrograma[i][0] == est and visitados.get(est) == None:
                    estAlcancaveis.append(self.fPrograma[i][2])
                    pilha.append(self.fPrograma[i][2])
            visitados[est] = True
                
        if not estAlcancaveis:
            estAlcancaveis = None
        else:
            estAlcancaveis = removeDuplicates(estAlcancaveis)
            estAlcancaveis.sort()
            
        return estAlcancaveis
        
        
    #Remove todos os estados inalcancaveis a partir do estado inicial
    def removeEstadosInalcancaveis(self):
        #converter para dicionacio
        estAting = self.estadosAlcancaveis(self.estadoInicial)
        estAting.append(self.estadoInicial)
        
        #tam = len(self.fPrograma)
        
        i = 0
        while i < len(self.fPrograma):
            # Se o i-esimo estado não é atingivel a partir do est inicial, remove ele e todas as suas ocorrências
            # da função programa
            if estAting.count(self.fPrograma[i][0]) == 0:
                self.fPrograma.pop(i)
                i -= 1
                #tam -= 1
            
            if estAting.count(self.fPrograma[i][2]) == 0:
                self.fPrograma.pop(i)
                i -= 1
                #tam -= 1
            i += 1
        
        #remove estados inatingiveis a partir do est inicial da lista de estados
        i = 0
        while i < len(self.estados):
        #for i in range(len(self.estados)):
            if estAting.count(self.estados[i]) == 0:
                self.estados.pop(i)
                i -= 1
            i += 1

            
    def funcaoPrograma(self, estado, simbolo):
        i = 0
        achou = False
        retorno = []
        
        while (not achou) and (i < len(self.fPrograma)):
            if (self.fPrograma[i][0] == estado) and (self.fPrograma[i][1] == simbolo):
                achou = True
                
                #Pega tds estados q tem msm estado e simbolo (afn)
                while (i < len(self.fPrograma)) and (self.fPrograma[i][0] == estado) and (self.fPrograma[i][1] == simbolo):
                    retorno.append(self.fPrograma[i][2])
                    i += 1 
            i += 1
        
        if not achou:
            retorno.append('&')
        
        return retorno
    
    
    def isFinal(self, estado):
        i = 0
        ret = True
        
        while i < len(self.estadosFinais):
            if self.estadosFinais[i] == estado:
                return True
            i += 1
        return False
    
    
    # Usando Mmin, avaliar uma lista de palavras, indicando quais são reconhecidas
    # por Mmin e quais são rejeitadas
    # obs: é essencial que no arquivo contendo o automato a função programa esteja ordenada
    # em ordem alfabetica pelo estado atual
    def verificaPalavras(self, wordList):
        if wordList: #se a lista de palavras nao esta vazia
            i = 1
            tam = len(self.fPrograma)
            resultado = {'aceitas':[], 'rejeitadas':[]}
            
            for word in wordList:
                aceita = False
                i = 1
                estadoAtual = []
                estadoAtual.append(self.estadoInicial)
                
                for letter in word:
                    estadoAtual.extend(self.funcaoPrograma(estadoAtual[0], letter))
                    estadoAtual.pop(0)
                    
                    #evita loop infinito em afn
                    if len(estadoAtual) > 1:
                        if estadoAtual[1] == estadoAtual[0]:
                            estadoAtual.pop(0)
                    
                    for estado in estadoAtual:
                        #print("Estado: ", estado)
                        if self.isFinal(estado) and i == len(word): #se for estado final e for ultima letra da palavra, aceita
                            resultado['aceitas'].append(word)
                            aceita = True
                    i += 1
                
                if aceita == False:
                    resultado['rejeitadas'].append(word)
                    
        return resultado

            
            
    #Dado um AFD M, gerar a GR equivalente (sera mandado arquivo ja aberto e dv ser fechado externamente a função (MODO APPEND!))
    def geraGR(self, fileOutput):
        fProducao = [[]] #contem simbolo variavel e prox simbolo
        
        if self.estadoInicial != '':
            tam = len(self.fPrograma) + len(self.estadosFinais) - 1 
            fProducao = [[0 for x in range(2)] for y in range(tam)]  #fProducao[tam][2]
            
            fProducao[0][0] = 'S'
            fProducao[0][1] = self.estadoInicial
            
            # Para tds estados finais, gera GR produzindo palavra vazia
            for i in range(len(self.estadosFinais)):
                fProducao[i+1][0] = self.estadosFinais[i]
                fProducao[i+1][1] = '&'
            
            # Percorre td função programa e converte em função de produção
            for j in range(len(self.fPrograma)):
                fProducao[i][0] = self.fPrograma[j][0]
                fProducao[i][1] = self.fPrograma[j][1] + self.fPrograma[j][2]
                i += 1
            
            #escreve definição da gramatica
            #fileOutput.write("G = ( {%s}, {%s}, S, P)\n" % (self.estados[i] for i in range(len(self.estados))),
            #                                               (fProducao[i][1] for i in range(len(self.estadosFinais) - 1, len(fProducao))))
            
            # G = ( {%s}, {%s}, S, P)\n
            print("G = ( {", end='', file=fileOutput)
            
            for i in range(len(self.estados)):
                print(self.estados[i], end='', file=fileOutput)
                
                if(i+1 < (len(self.estados))):
                    print(", ", end='', file=fileOutput)
            
            print("} , {", end='', file=fileOutput)
            
            for i in range(len(self.estadosFinais) - 1, len(fProducao)):
                print(fProducao[i][1], end='', file=fileOutput)
                
                if(i+1 < len(fProducao)):
                    print(", ", end='', file=fileOutput)
            
            print("}, S, P)", file=fileOutput)
            
            print("P", file=fileOutput)
            
            #escreve função de produção
            for k in range(len(fProducao)):
                fileOutput.write("%s -> %s\n" % (fProducao[k][0], fProducao[k][1]))
         
    def aMin(self):
        self = self.afnToAfd()
        self.removeEstadosInalcancaveis()
        self.convertAfdToFpt()
        
        return self.automatoMinimo()
    
    
    #Dado um AFN M, construir o seu equivalente autômato mínimo Mmin;
    #Ao chamar a função, tem q ser AFD!
    #tem q só ter estados alcançaveis via q0
    def automatoMinimo(self):
        #f programa: colunas = {estado atual, simbolo lido, prox estado}
        afm = Automato()
        pilha = []
        firstTime = True
        d = '&&'
        
        afm.estados = self.estados
        afm.estadoInicial = self.estadoInicial
        afm.fPrograma = self.fPrograma
        afm.alfabeto = self.alfabeto
        afm.estadosFinais = self.estadosFinais
        
        #Cria tabela de distincoes
        larg = len(self.estados)
        alt = larg
        tabDistincoes = [['-' for x in range(larg)] for y in range(alt)] #+1 para ser inclusivo no intervalo
        
        listaDependencias = {} #fazer com dicionario
        
        # Preenche matriz coluna 0 com estados
        for l in range(alt-1):
            tabDistincoes[l][0] = self.estados[l+1]
        l += 1 #armazena indice da ultima linha da tabela

        # Preenche ultima linha matriz com estados
        for a in range(1, larg):
            tabDistincoes[l][a] = self.estados[a-1]
        #tabDistincoes[l][a+1] = self.estados[a+1]
        
        
        # Marca pares estados fianis com não finais com true e o resto com false
        for i in range(0, l):
            for j in range(1, i+2): #j vai de 1 até i+1 (incluindo)
                if (self.isFinal(tabDistincoes[i][0])) and (not self.isFinal(tabDistincoes[l][j])):
                    tabDistincoes[i][j] = True
                elif (self.isFinal(tabDistincoes[l][j])) and (not self.isFinal(tabDistincoes[i][0])):
                    tabDistincoes[i][j] = True
                else:
                    tabDistincoes[i][j] = False
        
        
        # Percorre pares não marcados no passo anterior
        for i in range(0, l):
            for j in range(1, i+2):
                if tabDistincoes[i][j] == False:
                    p = tabDistincoes[i][0]
                    q = tabDistincoes[l][j]
                    
                    #para cada simbolo s do alfabeto, testar
                    for letter in self.alfabeto:
                        pR = self.funcaoPrograma(p, letter) #pResultante
                        qR = self.funcaoPrograma(q, letter)
                        pR = pR[0]
                        qR = qR[0]
                        
                        if pR == qR:
                            continue
                        
                        
                        #buscar indice do pR
                        ipR = 0
                        achou = False
                        
                        while not achou and ipR < l:
                            if tabDistincoes[ipR][0] == pR:
                                achou = True
                            else:
                                ipR += 1
                        if not achou:
                            continue
                        
                        #buscar indice do qR
                        iqR = 1
                        achou = False
                        k = 0
                        #k += 2 para evitar buscar fora limite tabela
                        #while not achou and iqR < k + 2 and k < l:
                        while not achou and iqR < larg:
                            if tabDistincoes[l][iqR] == qR:
                                achou = True
                            else:
                                iqR += 1
                                #k += 1
                        if not achou:
                            continue
                            
                        #if p == d or q == d or pR == d or qR == d:
                        #    continue
                        #se ipR ou iqR passarem dos limites, 'inverter' eles
                        if tabDistincoes[ipR][iqR] == '-':
                            tmp = ipR
                            ipR = iqR - 2
                            iqR = tmp + 2
                            
                            tmp = pR
                            pR = qR
                            qR = tmp
                            
                        
                        #"Se r e s não estão marcados, criar lista de dep com p e q encabeçados por r e s"
                        if tabDistincoes[ipR][iqR] == False: #mudar para p,q e pR, qR
                            listaDependencias[pR+','+qR] = []
                            listaDependencias[pR+','+qR].append([p,q,i,j]) #guarda p, q e o indice onde esta p,q
                        
                        #senao
                        #     tabDistincoes[p][q] == True
                        #     Se p,q possuem lista de dependencia, marcar tds estados dessa lista
                        
                        else:
                            #if p == d or q == d or pR == d or qR == d:
                            #    continue
                            
                            tabDistincoes[i][j] = True
                            
                            # Verifica se p,q possuem lista de dependencia, marcar tds estados dessa lista e
                            # verifica se cada estado marcado possui uma lista de dependencia (se sim, marcar tds)
                            # e assim recursivamente
                            # Pilha: guarda estados q foram marcados para verificar se possuem lista de dependencia
                            firstTime = True
                            while pilha or firstTime:
                                firstTime = False
                                    
                                if pilha:
                                    elem = pilha.pop()
                                    p1 = elem[0]
                                    q1 = elem[1]
                                else:
                                    p1 = p
                                    q1 = q
                            
                                if (listaDependencias.get(p+','+q) != None) or (listaDependencias.get(q+','+p) != None):
                                    # Se p,q possuem lista de dependencia, marcar tds estados dessa lista
                                    # e guardar estados marcados em uma pilha para verificar se possuem lista de dep tb
                                    if listaDependencias.get(p1+','+q1) != None:
                                        for listaEstados in listaDependencias[p1+','+q1]:
                                            tabDistincoes[listaEstados[2]][listaEstados[3]] = True
                                            pilha.append([listaEstados[0], listaEstados[1]]) #guarda estado marcado

                                            listaDependencias[p1+','+q1] = None

                                    if listaDependencias.get(q1+','+p1) != None:
                                        for listaEstados in listaDependencias[q1+','+p1]:
                                            tabDistincoes[listaEstados[2]][listaEstados[3]] = True
                                            pilha.append([listaEstados[0], listaEstados[1]])

                                        listaDependencias[q1+','+p1] = None     
        
        #Percorre tds estados não marcados e unifica eles
            #concatena dois estados q1 e q2 e forma um novo 'q1q2'
            #substitui tds transicoes q gerem q1 ou q2 por q1q2
            #transicoes desse novo estado são iguais as do q1 ou do q2 (pegar as transições de apenas 1 para ser as do q1q2)
            #exclui o outro estado do afd
        estadosUnificados = []
        #estadoU = ['EST_CONCAT', [ESTADOS UNIFICADOS]] (sempre que passar por um estado unificado, substituir pela sua concaten. (varredura fPrograma?))
        
        
        for i in range(0, l):
            for j in range(1, i+2):
                if tabDistincoes[i][j] == False:
                    if tabDistincoes[i][0] < tabDistincoes[l][j]:
                        estadoU = tabDistincoes[i][0] + tabDistincoes[l][j]
                    else:
                        estadoU = tabDistincoes[l][j] + tabDistincoes[i][0]
                    #estadosUnificados.append(tabDistincoes[i][0], tabDistincoes[l][j])
                    
                    #achou = False
                    
                    k = 0
                    #while not achou and k < len(self.fPrograma):
                    while k < len(afm.fPrograma):
                        # atualiza estados que foram atualizados
                        if afm.fPrograma[k][0] == tabDistincoes[i][0]:
                            afm.fPrograma[k][0] = estadoU
                        
                        if afm.fPrograma[k][2] == tabDistincoes[i][0]:
                            afm.fPrograma[k][2] = estadoU
                                               
                        # exclui o outro estado (pq foi unificado)
                        if self.fPrograma[k][0] == tabDistincoes[l][j]:
                            self.fPrograma.pop(k)
                            k -= 1
                            continue
                        
                        if self.fPrograma[k][2] == tabDistincoes[l][j]:
                            afm.fPrograma[k][2] = estadoU
                        
                        k += 1
                    
                    k = 0
                    
                    while k < len(afm.estados):
                        if afm.estados[k] == tabDistincoes[i][0]:
                            afm.estados[k] = estadoU
                        if afm.estados[k] == tabDistincoes[l][j]:
                            afm.estados.pop(k)
                            k -= 1
                            
                        k += 1
                    
                    k = 0
                    while k < len(afm.estadosFinais):
                        if afm.estadosFinais[k] == tabDistincoes[i][0]:
                            afm.estadosFinais[k] = estadoU
                        if afm.estadosFinais[k] == tabDistincoes[l][j]:
                            afm.estadosFinais.pop(k)
                            k -= 1
                            
                        k += 1
                    
                    if afm.estadoInicial == tabDistincoes[i][0]:
                        afm.estadoInicial = estadoU
                    
                    elif afm.estadoInicial == tabDistincoes[l][j]:
                        afm.estadoInicial = estadoU
        
        
        #Exclusão de estados inuteis
        visitado = {}
        pilha = []
        
        for state in afm.estados:
            visitado[state] = False
        
        pilha.append(afm.estadoInicial)
        
        while pilha:
            est = pilha.pop()
            visitado[est] = True
            est_fp = afm.estadosAtingiveis(est)
            #print("estAtingiveis: est ", est, ' - ', est_fp)
            #pprint.pprint(afm.estados)
            
            if est_fp == None:
                if not afm.isFinal(est): #se estado nao é final, achou estado inutil
                    i = 0
                    while i < len(afm.fPrograma):
                        if afm.fPrograma[i][0] == est or afm.fPrograma[i][2] == est:
                            afm.fPrograma.pop(i)
                            i -= 1
                        i += 1
                    #for i in range(len(afm.fPrograma)): #remove todas as ocorrencias dele na fPrograma
                    #    if afm.fPrograma[i][0] == est or afm.fPrograma[i][2] == est:
                    #        afm.fPrograma.pop(i)
                    #        i -= 1 #decrementa o i pq no for vai ser incrementado
                     
                    i = 0
                    while i < len(afm.estados):
                        if afm.estados[i] == est:
                            afm.estados.pop(i)
                            i -= 1
                        i += 1
                    #for i in range(len(afm.estados)):
                    #    if afm.estados[i] == est:
                    #        afm.estados.pop(i)
                    #        i -= 1
            else:
                # se estado atingido nao atinge um estado final, elimina ele
                if len(est_fp) == 1 and est_fp[0] == est and not afm.isFinal(est):
                    i = 0
                    while i < len(afm.fPrograma):
                        #print(afm.fPrograma)
                        if afm.fPrograma[i][0] == est or afm.fPrograma[i][2] == est:
                            afm.fPrograma.pop(i)
                            i -= 1
                        i += 1
                        
                    i = 0
                    while i < len(afm.estados):
                        if afm.estados[i] == est:
                            afm.estados.pop(i)
                            i -= 1
                        i += 1
                else:
                    for state in est_fp:
                        if not visitado[state]:
                            pilha.append(state)
            
            
        #prints
        '''for k in tabDistincoes:
            for e in k:
                print(e, ",\t", end='')
            print("\n")
        for m in afm.fPrograma:
            print(m)'''
            
        return afm

In [85]:
    # Dados dois AFD M1 e M2, decidir se ACEITA(M1) = ACEITA(M2)
    # Gerar afm
    # renomear tds estados para Qi
    # comparar estados alcancaveis a partir de q0. se forem iguais, linguagem é equivalente
    # fPrograma DEVE ESTAR ORDENADA!!!
    def verificaEquivalencia(m1, m2):
        minM1 = m1.aMin()
        #minM1 = m1.automatoMinimo()
        minM2 = m2.aMin()
        #pprint.pprint(minM1.fPrograma)
        #pprint.pprint(minM2.fPrograma)
        #minM1 = m1
        #minM2 = m2
        
        #verificar se cada estado do minM1 gera as mesmas transições que minM2
        
        if  (len(minM1.estados) != len(minM2.estados)) or (len(minM1.alfabeto) != len(minM2.alfabeto)) or (len(minM1.estadosFinais) != len(minM2.estadosFinais)):
                return False
        else:
            pilha = [] #[est_antigo, novo estado]
            index = 0
            #minM1.fPrograma.sort()
            #minM2.fPrograma.sort()
            
            #renomeia estados da função programa
            i = 0
            lastState = minM1.fPrograma[i][0]
            
            while i < len(minM1.fPrograma):
                pilha.append([minM1.fPrograma[i][0], '_Q' + str(index)])
                
                if minM1.fPrograma[i][0] == lastState:
                    lastState = minM1.fPrograma[i][0]
                    minM1.fPrograma[i][0] = '_Q' + str(index)
                
                else:
                    lastState = minM1.fPrograma[i][0]
                    index += 1
                    minM1.fPrograma[i][0] = '_Q' + str(index)
                i += 1
                
                      
            #renomeia estados restantes
            while pilha:
                tmp = pilha.pop()
                est_ant = tmp[0]
                est_renom = tmp[1]
                
                for i in range(len(minM1.fPrograma)):
                    if minM1.fPrograma[i][2] == est_ant:
                        minM1.fPrograma[i][2] = est_renom
                
                if minM1.estadoInicial == est_ant:
                    minM1.estadoInicial = est_renom
                        
                        
                
            #faz a msm coisa para m2
            pilha = []
            index = 0
            
            #renomeia estados da função programa
            i = 0
            lastState = minM2.fPrograma[i][0]
            
            while i < len(minM2.fPrograma):
                pilha.append([minM2.fPrograma[i][0], '_Q' + str(index)])
                
                if minM2.fPrograma[i][0] == lastState:
                    lastState = minM2.fPrograma[i][0]
                    minM2.fPrograma[i][0] = '_Q' + str(index)
                
                else:
                    lastState = minM2.fPrograma[i][0]
                    index += 1
                    minM2.fPrograma[i][0] = '_Q' + str(index)
                i += 1
            
            while pilha:
                tmp = pilha.pop()
                est_ant = tmp[0]
                est_renom = tmp[1]
                
                for i in range(len(minM2.fPrograma)):
                    if minM2.fPrograma[i][2] == est_ant:
                        minM2.fPrograma[i][2] = est_renom
                        
                if minM2.estadoInicial == est_ant:
                    minM2.estadoInicial = est_renom
                        
            #verifica se sao equivalentes
            #pilha1 = [], pilha2 = []
            #pilha1.append(minM1.estadoInicial), pilha2.append(minM2.estadoInicial)
            #print('renomeado')
            #pprint.pprint(minM1.fPrograma)
            #pprint.pprint(minM2.fPrograma)
            for i in range(len(minM1.fPrograma)):
                if minM1.fPrograma[i][0] != minM2.fPrograma[i][0] or minM1.fPrograma[i][2] != minM2.fPrograma[i][2]:
                    return False
            
            '''p = []
            visitados = {}
            p.append(minM1.estadoInicial)
            #while pilha1 and pilha2:
            while p:
                #e1 = pilha1.pop()
                #e2 = pilha2.pop()
                e = p.pop()
                
                print("minM1.estadosAlcancaveis(%s)" % e)
                am1 = minM1.estadosAlcancaveis(e)
                am2 = minM2.estadosAlcancaveis(e)
                
                print('am1 ', am1)
                print('am2 ', am2)
                
                if len(am1) != len(am2):
                    return False
                else:
                    for i in range(len(am1)):
                        if am1[i] != am2[i]:
                            return False
                        else:
                            if visitados.get(am1[i]) == None:
                                p.append(am1[i])
                                visitados[am1[i]] = True
                
            
            #print('am1 ', am1)
            #print('am1 ', am2)'''
            
        return True

In [86]:
#teste equivalencia

'''af = Automato()
af.alfabeto = ['0', '1']
af.estados = ['k', 'kk']
af.estadoInicial = 'k'
af.estadosFinais = ['kk']
af.fPrograma = [['k', '0', 'k'],
                 ['k', '1', 'kk'],
                 ['kk', '0', 'k'],
                 ['kk', '1', 'k'],]'''

af = Automato()
af.alfabeto = ['0', '1']
af.estados = ['q0', 'q1']
af.estadoInicial = 'q0'
af.estadosFinais = ['q1']
af.fPrograma = [['q0', '0', 'q0'],
                 ['q0', '1', 'q1'],
                 ['q1', '0', 'q1'],
                 ['q1', '1', 'q0']]

af2 = Automato()
af2.alfabeto = ['0', '1']
af2.estados = ['q0', 'q1', 'q2']
af2.estadoInicial = 'q0'
af2.estadosFinais = ['q2']
af2.fPrograma = [['q0', '0', 'q0'],
                 ['q0', '0', 'q1'],
                 ['q0', '1', 'q2'],
                 ['q1', '1', 'q2'],
                 ['q2', '0', 'q2'],
                 ['q2', '1', 'q1']]


'''t = af.afnToAfd()
t.removeEstadosInalcancaveis()
t.convertAfdToFpt()
pprint.pprint(t.fPrograma)
print('estados, ',t.estados)
print('ef, ',t.estadosFinais)
print('ei, ', t.estadoInicial)

print('----')

t2 = af2.afnToAfd()
t2.removeEstadosInalcancaveis()
t2.convertAfdToFpt()
pprint.pprint(t2.fPrograma)
print('estados, ',t2.estados)
print('ef, ',t2.estadosFinais)
print('ei, ', t2.estadoInicial)'''

#verificaEquivalencia(af,af2)


#pprint.pprint(a.fPrograma)
#print('True') if verificaEquivalencia(af,af2) else print("False")
#k = af.automatoMinimo()
print('True') if verificaEquivalencia(af,af2) else print("False")

True


In [45]:
#teste afn

'''af = Automato()
af.alfabeto = ['0', '1']
af.estados = ['k', 'kk']
af.estadoInicial = 'k'
af.estadosFinais = ['kk']
af.fPrograma = [['k', '0', 'k'],
                 ['k', '1', 'kk'],
                 ['kk', '0', 'k'],
                 ['kk', '1', 'k'],]'''

af = Automato()
af.alfabeto = ['0', '1']
af.estados = ['q0', 'q1']
af.estadoInicial = 'q0'
af.estadosFinais = ['q1']
af.fPrograma = [['q0', '0', 'q0'],
                 ['q0', '1', 'q1'],
                 ['q1', '0', 'q0'],
                 ['q1', '1', 'q0']]

af2 = Automato()
af2.alfabeto = ['0', '1']
af2.estados = ['q0', 'q1', 'q2']
af2.estadoInicial = 'q0'
af2.estadosFinais = ['q2']
af2.fPrograma = [['q0', '0', 'q0'],
                 ['q0', '0', 'q1'],
                 ['q0', '1', 'q2'],
                 ['q1', '1', 'q2'],
                 ['q2', '0', 'q2'],
                 ['q2', '1', 'q1']]


#t = af.afnToAfd()
#t = af2.convertAfdToFpt()
#a = af2.afnToAfd()
#pprint.pprint(a.fPrograma)
#print('True') if verificaEquivalencia(af,af2) else print("False")
#k = af.automatoMinimo()
k = af2.afnToAfd()
pprint.pprint(k.fPrograma)
print('estados, ',k.estados)
print('ef, ',k.estadosFinais)

print('ei, ', k.estadoInicial)
#k.removeEstadosInalcancaveis()
#k.convertAfdToFpt()
#k2 = k.automatoMinimo()
#pprint.pprint(k.fPrograma)
#pprint.pprint(k.estados)

'''
self = self.afnToAfd()
        self.removeEstadosInalcancaveis()
        self.convertAfdToFpt()'''

[['Q0', '0', 'Q1'],
 ['Q0', '1', 'Q2'],
 ['Q1', '0', 'Q1'],
 ['Q1', '1', 'Q2'],
 ['Q2', '0', 'Q2'],
 ['Q2', '1', 'Q1']]
estados,  ['Q0', 'Q1', 'Q2']
ef,  ['Q2']
ei,  Q0


'\nself = self.afnToAfd()\n        self.removeEstadosInalcancaveis()\n        self.convertAfdToFpt()'

In [7]:
listaMaiorTamanho([
    [2,4,5],
    [2,4,5,6,7,8],
    [2,4,5,6,7,8,9,9],
    [2,4],
    [2,4,5,1,2]
])

[2, 4, 5, 6, 7, 8, 9, 9]

In [8]:
#teste afm


import pprint
af2 = Automato()
af2.alfabeto = ['a', 'b']
#af2.estados = ['q0', 'q1', 'q2', 'q3', 'q4', 'q5']
af2.estados = ['q0', 'q1', 'q2', 'q3', 'q4', 'q5']
af2.estadoInicial = 'q0'
af2.estadosFinais = ['q0', 'q4', 'q5']
af2.fPrograma = [['q0', 'a', 'q2'],
                 ['q0', 'b', 'q1'],
                 ['q1', 'a', 'q1'],
                 ['q1', 'b', 'q0'],
                 ['q2', 'a', 'q4'],
                 ['q2', 'b', 'q5'],
                 ['q3', 'a', 'q5'],
                 ['q3', 'b', 'q4'],
                 ['q4', 'a', 'q3'],
                 ['q4', 'b', 'q2'],
                 ['q5', 'a', 'q2'],
                 ['q5', 'b', 'q3']]

pprint.pprint(af2.fPrograma)
af3 = af2.automatoMinimo()
print(af3.estados)
pprint.pprint(af2.fPrograma)



#for s in af2.fPrograma:
#    print(s)

#af2.automatoMinimo()    
#af2.estados
#print(af2.estados)
#print(af2.fPrograma)

[['q0', 'a', 'q2'],
 ['q0', 'b', 'q1'],
 ['q1', 'a', 'q1'],
 ['q1', 'b', 'q0'],
 ['q2', 'a', 'q4'],
 ['q2', 'b', 'q5'],
 ['q3', 'a', 'q5'],
 ['q3', 'b', 'q4'],
 ['q4', 'a', 'q3'],
 ['q4', 'b', 'q2'],
 ['q5', 'a', 'q2'],
 ['q5', 'b', 'q3']]


AttributeError: 'Automato' object has no attribute 'automatoMinimo'

In [ ]:
#teste alcancavel

import pprint
af2 = Automato()
af2.alfabeto = ['a', 'b']
#af2.estados = ['q0', 'q1', 'q2', 'q3', 'q4', 'q5']
af2.estados = ['q0', 'q1', 'q2', 'q3', 'q4', 'q5', 'q6', 'q7']
af2.estadoInicial = 'q0'
af2.estadosFinais = ['q0', 'q4', 'q5']
af2.fPrograma = [['q0', 'a', 'q2'],
                 ['q0', 'b', 'q1'],
                 #['q1', 'a', 'q1'],
                 ['q1', 'a', 'q1'],
                 ['q1', 'b', 'q0'],
                 ['q2', 'a', 'q4'],
                 ['q2', 'b', 'q5'],
                 ['q3', 'a', 'q5'],
                 ['q3', 'b', 'q4'],
                 ['q4', 'a', 'q3'],
                 ['q4', 'b', 'q2'],
                 ['q5', 'a', 'q2'],
                 ['q5', 'b', 'q3'],
                 ['q6', 'a', 'q6'],
                 ['q6', 'b', 'q6'],
                 ['q7', 'b', 'q3']
                ]
'''print(af2.estadosAlcancaveis('q0'))
print("\n", end='')
pprint.pprint(af2.fPrograma)
print("\n", end='')
af2.removeEstadosInalcancaveis()
pprint.pprint(af2.fPrograma)'''

af2.convertAfdToFpt()
pprint.pprint(af2.fPrograma)





#af2.estadosAtingiveis('q0')

#af2.removeEstadosInatingiveis()

#print(af2.fPrograma)

#af2.convertAfdToFpt()

#for s in af2.fPrograma:
#    print(s)

#af2.automatoMinimo()    
#af2.estados
#print(af2.estados)
#print(af2.fPrograma)

In [ ]:
af2 = Automato()
af2.alfabeto = ['0', '1']
af2.estados = ['A', 'B', 'C']
af2.estadoInicial = 'A'
af2.estadosFinais = ['C']
af2.fPrograma = [['A', '0', 'A'],
                 ['A', '0', 'B'],
                 ['A', '1', 'A'],
                 ['B', '0', 'C']
                ]
af3 = af2.afnToAfd()
print("fprograma:")
#for k in af3.fPrograma:
#            print(k)            
pprint.pprint(af3.fPrograma)
print("est finais: ",af3.estadosFinais)
print("estados: ", af3.estados)
print("alfabeto: ", af3.alfabeto)
print("estado inicial: ", af3.estadoInicial)

In [ ]:
### TESTES ###
#af = txtToAutomato('teste.txt')
af2 = Automato()
af2.alfabeto = ['a', 'b']
af2.estados = ['q0', 'q1', 'q2', 'q3', 'q4', 'q5', 'q6', 'q7']
af2.estadoInicial = 'q0'
af2.estadosFinais = ['q2']
af2.fPrograma = [['q0', 'a', 'q1'],
                 ['q0', 'b', 'q5'],
                 ['q1', 'a', 'q6'],
                 ['q1', 'b', 'q2'],
                 ['q2', 'a', 'q0'],
                 ['q2', 'b', 'q2'],
                 ['q3', 'a', 'q2'],
                 ['q4', 'a', 'q7'],
                 ['q4', 'b', 'q5'],
                 ['q5', 'a', 'q2'],
                 ['q5', 'b', 'q6'],
                 ['q6', 'b', 'q4'],
                 ['q7', 'a', 'q6'],
                 ['q7', 'b', 'q2']
                ]

af3 = af2.afnToAfd()
#print(af3.estados)
#af3.convertAfdToFpt()

for k in af3.fPrograma:
    print(k) 
#print(af3.estados)
    
#af2.automatoMinimo()


In [ ]:
def Remove(duplicate): 
    final_list = [] 
    for num in duplicate: 
        if num not in final_list: 
            final_list.append(num) 
    return final_list 
      
# Driver Code 
duplicate = ['a', 'b', 'q0', 'a', 'q0', 'q1'] 
print(Remove(duplicate)) 


In [ ]:
x = [['a', 'l', 'o']]
x.append(['', '', 't'])
print(x[1][2])
print(x)

In [ ]:
x = [['q0', 'a', 'q1'],
 ['q0', 'b', 'q5'],
 ['q1', 'a', 'q6'],
 ['q1', 'b', 'q2'],
 ['q2', 'a', 'q0'],
 ['q2', 'b', 'q2']
]

x.insert(2, ["TESTE!", '', ''])
for s in x:
    print(s)

In [ ]:
x = ['AB', 'A', 'ABC']
x.sort()
print(x)

In [ ]:
['q1', 0   , 0   , 0   , 0],
['q2', 0   , 0   , 0   , 0], 
['q3', True, 0   , 0   , 0], 
['d' , 0   , True, 0   , 0], 
[0   , 'q0', 'q1', 'q2', 'q3']

In [ ]:
#outFilename = input("Insira o nome do arquivo: ")
outFilename = "testeGr"
outFile = open(outFilename+'.txt', 'w')
af.geraGR(outFile)
outFile.close()

In [ ]:
#TESTES
af = txtToAutomato('teste.txt')
print(af.alfabeto)
print(af.estados)
print(af.estadoInicial)
print(af.estadosFinais)
print(af.fPrograma)
print("\n")
print(af.funcaoPrograma('q2','a'))
print(af.funcaoPrograma('q1','a'))
print(af.funcaoPrograma('q0','a'))
print("\n")
print('True') if af.isFinal('q3') else print('False')
print("\n")
wordList = []

wordList.append("b")
wordList.append("a")
wordList.append("ba")
wordList.append("aa")
wordList.append("bab")
wordList.append("baba")
res = af.verificaPalavras(wordList)
print(*wordList)
print("Aceitas: ", *res['aceitas'])
print("Rejeitadas:", *res['rejeitadas'])
print("\n")

In [ ]:
##ARQUIVO TESTE
AUTÔMATO=({q0,q1,q2,q3},{a,b},q0,{q1,q3})
Prog
(q0,a)=q1
(q0,b)=q2
(q1,b)=q2
(q2,a)=q3
(q2,a)=q2
(q3,a)=q3
(q3,b)=q2


In [ ]:
teste = {}
teste['t1'] = 1
teste['t2'] = 2
teste['t3'] = 3
teste['t4'] = 4

#print(teste['t4'])

print('true') if teste.get('t5') != None else print('false')

In [ ]:
x = re.findall('{.+}', '({q0,q1,q2,q3},{a,b},q0,{q1,q3})')
print(x)

In [ ]:
#print("t", end='')
#print("e", end='')

pile = []
pile.append(['t','e'])
pile.pop()
print("vazia") if not pile else print("nao vazia")

In [ ]:
file = open('teste.txt', 'r')
content = file.readlines()
file.close()
x = re.split("=", content[0])
k = re.findall('({[A-Za-z0-9,]+})',x[1])

k = re.findall('(},[A-Za-z0-9]+,{)', x[1])
k = re.findall('([A-Za-z0-9]+)', k[0])
print(k)

In [ ]:
import re
x = []

x = re.split(',','{q0,q1,q2,q3}')
print(x)

In [ ]:
re.findall('[A-Za-z0-9]+','{q0a,q1,q2,q3}')

In [ ]:
import re

#Check if the string starts with "The" and ends with "Spain":

txt = "AUTÔMATO=({q0,q1,q2,q3},{a,b},q0,{q1,q3})"
txt2 = "({q0,q1,q2,q3},{a,b},q0,{q1,q3})"
x = re.split("=", txt)

print(x)
print(txt)

y = re.findall('([A-Za-z]+)', 'tes22te')

#y = re.split("({[A-Z0-9]+})", y[0]) #{q0,q1,q2,q3},{a,b},q0,{q1,q3}
print(y)
